In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [2]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars = sorted(set(text))
vocabulary_size = len(chars)
print(chars)
print(vocabulary_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda w: [string_to_int[c] for c in w]
decode = lambda n: ''.join([int_to_string[i] for i in n])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)

tensor([80,  1,  1, 28, 39, 42, 39, 44])
tensor([ 1,  1, 28, 39, 42, 39, 44, 32])


In [8]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, target is {target}.")

When input is tensor([80]), target is 1.
When input is tensor([80,  1]), target is 1.
When input is tensor([80,  1,  1]), target is 28.
When input is tensor([80,  1,  1, 28]), target is 39.
When input is tensor([80,  1,  1, 28, 39]), target is 42.
When input is tensor([80,  1,  1, 28, 39, 42]), target is 39.
When input is tensor([80,  1,  1, 28, 39, 42, 39]), target is 44.
When input is tensor([80,  1,  1, 28, 39, 42, 39, 44]), target is 32.


In [9]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs', x.shape, x)
print('targets', y.shape, y)

tensor([ 76160,  13744, 157870,   4370])
inputs torch.Size([4, 8]) tensor([[72, 68,  1, 66, 54, 67, 78,  1],
        [58,  1, 68, 71,  1, 54, 67,  0],
        [62, 65, 68, 74, 72,  0, 54, 57],
        [ 1,  1,  1,  1,  1,  1,  1, 16]])
targets torch.Size([4, 8]) tensor([[68,  1, 66, 54, 67, 78,  1, 75],
        [ 1, 68, 71,  1, 54, 67,  0, 74],
        [65, 68, 74, 72,  0, 54, 57, 75],
        [ 1,  1,  1,  1,  1,  1, 16, 13]])


In [ ]:
class BigramLanguageModel(nn.Module):
    pass